In [ ]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import community.community_louvain as community_louvain
import funkcie as fn
fn.setup_directories()

#### NAČÍTANIE DÁT

In [ ]:
G = fn.load_data("facebook_combined.txt")
pos = nx.spring_layout(G, seed=42)
fn.draw_graph_raw(G, pos)

Načítavam dáta z: facebook_combined.txt
-> Uzlov: 4039
-> Hrán: 88234
Počítam layout (môže to chvíľu trvať)...


#### VYHODNOTENIE

In [5]:
# Zozbieranie výsledkov do zoznamu
results = [
    {"Model": "Louvain", "Modularita": round(mod_louvain, 4), "Počet komunít": count_louvain},
    {"Model": "LPA",     "Modularita": round(mod_lpa, 4),     "Počet komunít": count_lpa}
]

# Uloženie a zobrazenie tabuľky
df_results = fn.save_results_table(results)
df_results

,Model,Modularita,Počet komunít
0,Louvain,0.8349,15
1,LPA,0.7368,44


#### EXPERIMENT A: Vlpyv hyperparametra resolution na výsledky modelu Louvain

In [26]:
# ==========================================
# EXPERIMENT B: Hĺbková analýza víťazného modelu (Louvain)
# ==========================================
import matplotlib.pyplot as plt
import pandas as pd
import collections
import community.community_louvain as community_louvain
import funkcie as fn 

# 1. Načítanie najlepšieho modelu (z Experimentu A)
# Použijeme resolution, ktoré vyhralo v predchádzajúcom kroku
try:
    final_res = best_resolution
except NameError:
    final_res = 1.0

# Výpočet finálneho rozdelenia pre Louvain
final_partition = community_louvain.best_partition(G, resolution=final_res, random_state=42)

# Získanie veľkostí a zoradenie od najväčšej po najmenšiu
community_counts = collections.Counter(final_partition.values())
sorted_sizes = sorted(community_counts.values(), reverse=True)

# -------------------------------------------------------
# 2. Uloženie GRAFU (Zoradené veľkosti - "Rank-Size Plot")
# -------------------------------------------------------
plt.figure(figsize=(10, 6))

# Vytvoríme x-ovú os ako poradie (1, 2, 3...)
ranks = range(1, len(sorted_sizes) + 1)

# Vykreslenie stĺpcového grafu
bars = plt.bar(ranks, sorted_sizes, color='tab:green', edgecolor='black', alpha=0.75)

plt.title(f"Veľkosť komunít zoradená podľa poradia (Louvain, res={final_res})")
plt.xlabel("Poradie komunity (od najväčšej po najmenšiu)")
plt.ylabel("Počet členov (uzlov)")

# Nastavenie X-ovej osi
if len(ranks) <= 20:
    plt.xticks(ranks)
else:
    plt.locator_params(axis='x', integer=True)

plt.grid(axis='y', linestyle='--', alpha=0.5)

# Pridanie hodnoty nad prvý (najväčší) stĺpec pre efekt
plt.text(ranks[0], sorted_sizes[0], f" {sorted_sizes[0]}", ha='center', va='bottom', fontweight='bold')

# Uloženie s názvom Experiment B
plt.savefig("img/experiment_B_sizes_sorted.png", dpi=300, bbox_inches='tight')
plt.close()

# -------------------------------------------------------
# 3. Uloženie TABUĽKY "Top 5 Komunít"
# -------------------------------------------------------
# Pripravíme dáta pre Top 5
sorted_comms_with_ids = sorted(community_counts.items(), key=lambda x: x[1], reverse=True)

top_5_data = []
total_nodes = G.number_of_nodes()

for i, (comm_id, size) in enumerate(sorted_comms_with_ids[:5]):
    share = (size / total_nodes) * 100
    top_5_data.append([
        str(i + 1),               # Poradie
        str(comm_id),             # ID Komunity
        str(size),                # Počet členov
        f"{share:.1f} %"          # Podiel na sieti
    ])

# Vytvorenie obrázka pre tabuľku
fig_tbl, ax_tbl = plt.subplots(figsize=(6, 2.5))
ax_tbl.axis('tight')
ax_tbl.axis('off')

columns = ["Poradie", "ID Komunity", "Počet členov", "Podiel na sieti"]

table = ax_tbl.table(cellText=top_5_data,
                     colLabels=columns,
                     cellLoc='center',
                     loc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

plt.title("Top 5 najväčších komunít (Louvain)", y=1.0, pad=20, fontsize=12, fontweight='bold')

# Uloženie s názvom Experiment B
plt.savefig("img/experiment_B_top5.png", dpi=300, bbox_inches='tight')
plt.close()